In [78]:
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import pandas as pd

In [79]:
# driver = webdriver.Chrome()
driver = webdriver.Edge()

driver.get("https://investor.vanguard.com/investment-products/list/all?filters=open")
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
div_table = soup.find("div", id="assetAllocationTableExportDiv")
table = div_table.find("table")

In [34]:
cells = table.find('thead').find_all('tr')[1].find_all('th')
headers = [cell.get_text(strip=True) for cell in cells]
headers

['Symbol|Name',
 'Asset class',
 'Risk',
 'Expenseratio',
 'Compare',
 'Transact',
 'SECyield',
 'YTD',
 '1-yr',
 '5-yr',
 '10-yr',
 'Sinceinception',
 'InvestmentMinimum',
 'Price',
 'Change']

In [17]:
tr = table.find("tbody").find_all("tr")

In [15]:
tr[0].find("span", attrs={"data-rpa-tag-id": "symbol"}).get_text(strip=True)

'VBIL'

In [16]:
tr[0].find("a", attrs={"data-rpa-tag-id": "longName"}).get_text(strip=True)

'0-3 Month Treasury Bill ETF'

In [19]:
tr[0].find("p", attrs={"data-rpa-tag-id": "expenseRatio"}).get_text(strip=True)

'0.07%'

In [88]:
data = []

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")            
    div_table = soup.find("div", id="assetAllocationTableExportDiv")
    table = div_table.find("table")
        
    for tr in table.find("tbody").find_all("tr"):
        symbol = tr.find("span", attrs={"data-rpa-tag-id": "symbol"}).get_text(strip=True)
        fund_name = tr.find("a", attrs={"data-rpa-tag-id": "longName"}).get_text(strip=True)
        er = tr.find("p", attrs={"data-rpa-tag-id": "expenseRatio"}).get_text(strip=True).rstrip('%')        
        data.append({"symbol": symbol, "fund_name": fund_name, "expense_ratio": er})
    
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'button[title="next page"]')        
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
        time.sleep(3)      
        next_button.click()       
    except:
        print("No next button found")
        break


No next button found


In [89]:
data

[{'symbol': 'VBIL',
  'fund_name': '0-3 Month Treasury Bill ETF',
  'expense_ratio': '0.07'},
 {'symbol': 'VFIAX',
  'fund_name': '500 Index Fund Admiral Shares',
  'expense_ratio': '0.04'},
 {'symbol': 'VADGX',
  'fund_name': 'Advice Select Dividend Growth Fund',
  'expense_ratio': '0.41'},
 {'symbol': 'VAGVX',
  'fund_name': 'Advice Select Global Value Fund',
  'expense_ratio': '0.40'},
 {'symbol': 'VAIGX',
  'fund_name': 'Advice Select International Growth Fund',
  'expense_ratio': '0.40'},
 {'symbol': 'VBPIX',
  'fund_name': 'Baillie Gifford Global Positive Impact Stock Fund Investor Shares',
  'expense_ratio': '0.59'},
 {'symbol': 'VBIAX',
  'fund_name': 'Balanced Index Fund Admiral Shares',
  'expense_ratio': '0.07'},
 {'symbol': 'VBAIX',
  'fund_name': 'Balanced Index Fund Institutional Shares',
  'expense_ratio': '0.06'},
 {'symbol': 'VCADX',
  'fund_name': 'California Intermediate-Term Tax-Exempt Fund Admiral Shares',
  'expense_ratio': '0.09'},
 {'symbol': 'VCAIX',
  'fund_na

In [76]:
driver.find_element(By.CSS_SELECTOR, 'button[title="next page"]').click()

In [90]:
driver.close()

In [91]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,symbol,fund_name,expense_ratio
0,VBIL,0-3 Month Treasury Bill ETF,0.07
1,VFIAX,500 Index Fund Admiral Shares,0.04
2,VADGX,Advice Select Dividend Growth Fund,0.41
3,VAGVX,Advice Select Global Value Fund,0.40
4,VAIGX,Advice Select International Growth Fund,0.40


In [95]:
df.to_parquet("vanguard_info.parquet", index=False)